In [43]:
import os
import openai
import langchain
import pypdf
import pickle
from llama_index import download_loader
from langchain.llms import AzureOpenAI
from langchain.document_loaders import PyPDFLoader,CSVLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chat_models import AzureChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.chains import ConversationalRetrievalChain

# Constants
BASE_URL = "https://gpt-demo1.openai.azure.com"
API_TYPE = "azure"
API_VERSION = "2023-03-15-preview"
DEPLOYMENT_NAME = "GPT35-Demo1" #other names: "text-davinci-003"
MODEL_NAME = "gpt-35-turbo" #other names: "text-davinci-003"
API_FILE = "/home/azureuser/cloudfiles/code/keys/openai_key.txt"

In [44]:
openai.api_type = "azure"
openai.api_base = "https://gpt-demo1.openai.azure.com/"
openai.api_version = "2022-12-01"

# openai.api_key = os.getenv("OPENAI_API_KEY")

with open("/home/azureuser/cloudfiles/code/keys/openai_key.txt", 'r') as file:
   api_key = file.read().replace('\n', '')

openai.api_key = api_key
os.environ["OPENAI_API_KEY"] = api_key
os.environ["AZURE_OPENAI_API_KEY"] = api_key
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://gpt-demo1.openai.azure.com/"

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

os.environ["OPENAI_ENGINES"] = "GPT-35-Demo1"
os.environ["OPENAI_EMBEDDINGS_ENGINE_DOC"] = "text-embedding-ada"
os.environ["OPENAI_EMBEDDINGS_ENGINE_QUERY"] = "text-embedding-ada"
os.environ["OPENAI_API_BASE"] = "https://gpt-demo1.openai.azure.com"
os.environ["OPENAI_ENGINES"] = "GPT35-Demo1"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"


In [45]:
def gpt_init():
    #llm = AzureOpenAI(model_name=MODEL_NAME)
    #llm = ChatOpenAI(temperature=0.7, model_name=MODEL_NAME, engine=DEPLOYMENT_NAME)
    #llm.deployment_name = DEPLOYMENT_NAME
    #llm.openai_api_base = BASE_URL
    #llm.openai_api_key = os.getenv("OPENAI_API_KEY")
    #llm.max_tokens = 512

    model = AzureChatOpenAI(
        openai_api_base=BASE_URL,
        openai_api_version=API_VERSION,
        deployment_name=DEPLOYMENT_NAME,
        openai_api_key=API_KEY,
        openai_api_type = API_TYPE,
    )
    return model

In [4]:
question = "What is physicians brain digital twin?"
#llm = OpenAI(model_name="text-davinci-003",temperature=0.9)

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003")
#print(llm(question))

#llm = ChatOpenAI(temperature=0,model_name="gpt-4")


'''
#pdf loader
loader = PyPDFLoader("database/cisco_1.pdf")
pages = loader.load_and_split()
print(pages)

#text loader
loader = DirectoryLoader('database', glob="**/*.txt")
pages = loader.load()
#print(pages)

#csv loader
loader = CSVLoader(file_path='database/GrammarFile.csv')
pages = loader.load()
data = [Document(page_content=str(pages))]

#html loader
loader = UnstructuredHTMLLoader("Users/sri_raguram/database/cisco2023-6.pdf")
data = loader.load()





'''

#URL Loader
urls = [
    "https://www.dissentmagazine.org/online_articles/russia-war-with-no-end"
]
loader = UnstructuredURLLoader(urls=urls)
pages = loader.load()




merged_db = FAISS.from_documents([Document(page_content=" ")], OpenAIEmbeddings(model="text-embedding-ada-002"))

search_indices = []
for i, chunk in enumerate(pages):
    data = [Document(page_content=str(chunk))]
    var_name = f"search_index_{i}"
    locals()[var_name] = FAISS.from_documents(data, OpenAIEmbeddings(model="text-embedding-ada-002"))
    search_indices.append(locals()[var_name])
    #merged_db.merge_from(locals()[var_name])


for db in search_indices:
    merged_db.merge_from(db)

embeddings = OpenAIEmbeddings()
with open("vectorstore.pkl", "wb") as f:
        pickle.dump(merged_db, f)

with open("vectorstore.pkl", "rb") as f:
  vectorstore = pickle.load(f)

while True:
    query = input("Enter Prompt: ")
    qa = ConversationalRetrievalChain.from_llm(AzureChatOpenAI(deployment_name="GPT35-Demo1", model_name="gpt-35-turbo", temperature=0), vectorstore.as_retriever())
    response = qa({"question": query, "chat_history": ""})
    print(response["answer"])


USING API_BASE: 
https://gpt-demo1.openai.azure.com
The article was published on April 28, 2023.
The author is Sasha de Vogel.


In [1]:
type_input = input("Enter Document Type(pdf,txt,csv,html): ").lower()
if type_input == "pdf":
    loader = PyPDFLoader("database/Cisco_Sample.pdf")
    pages = loader.load_and_split()
elif type_input == "txt":
    loader = DirectoryLoader('database', glob="**/*.txt")
    pages = loader.load()
elif type_input == "csv":
    loader = CSVLoader(file_path='database/GrammarFile.csv')
    data = loader.load()
    pages = [Document(page_content=str(data))]
elif type_input == "html":
    loader = UnstructuredHTMLLoader("database/sample.html")
    pages = loader.load()
else:
    print("Choose Appropriate Document Type!")
while True:
    query_input = input("Enter Prompt: ")
    llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003")
    print("ChatGPT without Source!")
    print("-------------------------------------------------------------")
    print(llm(query_input))
    print("-------------------------------------------------------------")
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(pages, embeddings)
    with open("vectorstore.pkl", "wb") as f:
            pickle.dump(vectorstore, f)
    qa = ConversationalRetrievalChain.from_llm(AzureChatOpenAI(deployment_name="GPT35-Demo1", model_name="gpt-35-turbo", temperature=0), vectorstore.as_retriever())
    response = qa({"question": query_input, "chat_history": ""})
    print("ChatGPT without Source!")
    print("-------------------------------------------------------------")
    print(response["answer"])
    print("-------------------------------------------------------------")








In [12]:

!pip install pdfminer

  Using cached pdfminer-20191125-py3-none-any.whl


In [56]:
question = "What is physicians brain digital twin?"
#llm = OpenAI(model_name="text-davinci-003",temperature=0.9)

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003")
#print(llm(question))

#llm = ChatOpenAI(temperature=0,model_name="gpt-4")


'''
#pdf loader
loader = PyPDFLoader("database/cisco_1.pdf")
pages = loader.load_and_split()
print(pages)

#text loader
loader = DirectoryLoader('database', glob="**/*.txt")
pages = loader.load()
#print(pages)

#csv loader
loader = CSVLoader(file_path='database/GrammarFile.csv')
pages = loader.load()
data = [Document(page_content=str(pages))]

#html loader
loader = UnstructuredHTMLLoader("Users/sri_raguram/database/cisco2023-6.pdf")
data = loader.load()





'''


SimpleDirectoryReader = download_loader("SimpleDirectoryReader")
loader = SimpleDirectoryReader('database', recursive=True, exclude_hidden=True)
page = str(loader.load_data())
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
pagess = text_splitter.split_text(page)



merged_db = FAISS.from_documents([Document(page_content=" ")], OpenAIEmbeddings(model="text-embedding-ada-002"))

search_indices = []
for i, chunk in enumerate(pagess):
    data = [Document(page_content=str(chunk))]
    var_name = f"search_index_{i}"
    locals()[var_name] = FAISS.from_documents(data, OpenAIEmbeddings(model="text-embedding-ada-002"))
    search_indices.append(locals()[var_name])
    #merged_db.merge_from(locals()[var_name])


for db in search_indices:
    merged_db.merge_from(db)

embeddings = OpenAIEmbeddings()
with open("vectorstore.pkl", "wb") as f:
        pickle.dump(merged_db, f)


with open("vectorstore.pkl", "rb") as f:
  vectorstore = pickle.load(f)

while True:
    query = input("Enter Prompt: ")
    qa = ConversationalRetrievalChain.from_llm(AzureChatOpenAI(deployment_name="GPT35-Demo1", model_name="gpt-35-turbo", temperature=0), vectorstore.as_retriever())
    response = qa({"question": query, "chat_history": ""})
    print(response["answer"])


USING API_BASE: 
https://gpt-demo1.openai.azure.com
The author name of cisco_brt is sriraguram.


[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...


In [53]:
for i in pagess:
    print('-----------------------------------------------')
    print(i)

-----------------------------------------------
[Document(text="HospitalID: 1004176\nHospitalName: Malik Radix Health Care\nGrammar: PRIMARY_DIAGNOSIS@[{'LOWER':'diagnosis'}];EXIT()@[{'TEXT':'PHYSICAL'},{'TEXT':'COMPLAINTS'}]::SECONDARY_DIAGNOSIS@[{'TEXT':'PHYSICAL'},{'TEXT':'COMPLAINTS'}];EXIT()@[{'TEXT':'CLINICAL'},{'TEXT':'EXAMINATION'}]::INVESTIGATIONS@[{'TEXT':'INVESTIGATION'}];EXIT()@[{'TEXT':'TREATMENT'},{'TEXT':'GIVEN'}]", doc_id='f9dd6c2b-fa19-4247-8ee8-026a23e9ca9f', embedding=None, doc_hash='9a8aa99a5fb8760a1fe65f1a91f17540f7f121c857745079572a1cfc11d8ab89', extra_info=None), Document(text="HospitalID: 1264648\nHospitalName: Pratiksha Hospital(North East Healthcare Pvt Ltd)\nGrammar: DIAGNOSIS@[{'TEXT':'DIAGNOSIS'},{'IS_PUNCT':True}];EXIT()@[{'TEXT':'ON'},{'TEXT':'EXAMINATION'}]::SECONDARY_DIAGNOSIS@[{'TEXT':'PRESENTING'},{'TEXT':'COMPLAINTS'},{'IS_PUNCT':True}];EXIT()@[{'TEXT':'PAST'},{'TEXT':'HISTORY'}]::INVESTIGATIONS@[{'TEXT':'SIGNIFICANT'},{'TEXT':'FINDINGS'}];EXIT()@[{'